# import libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime,date
import geopandas as gpd
from geopandas import GeoDataFrame as gdf
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from shapely.geometry import Point
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime

import glob
import os

# load data

In [3]:
raw_data = pd.read_csv('..\..\Raw GPS data Kandy Buses\digana_2022_09.csv')
bus_terminals= pd.read_csv('bus_stops_and_terminals_654.csv')

In [4]:
raw_data

,id,deviceid,devicetime,latitude,longitude,speed
0,1369679821,264,2022-09-01 00:48:29,7.296963,80.744453,0.0000
1,1369679822,264,2022-09-01 00:33:29,7.296963,80.744453,0.0000
2,1369679823,264,2022-09-01 00:18:29,7.296963,80.744453,0.0000
3,1369679824,264,2022-09-01 00:03:29,7.296963,80.744453,0.0000
4,1369679825,264,2022-08-31 23:48:29,7.296963,80.744453,0.0000
...,...,...,...,...,...,...
634241,1456577102,123,2022-10-01 23:59:11,7.198013,80.298893,16.7387
634242,1456577103,123,2022-10-01 23:58:57,7.197230,80.298810,10.2592
634243,1456577104,123,2022-10-01 23:58:42,7.197228,80.299745,15.6588
634244,1456577105,123,2022-10-01 23:58:27,7.196645,80.300475,10.2592


In [5]:
# df = raw_data[raw_data['deviceid']==123]
# df['devicetime'] = pd.to_datetime(df['devicetime'])
# df = df[(df['devicetime'] >= '2021-10-04 11:01:04') & (df['devicetime'] <= '2021-10-04 11:50:47')]
# df

# data cleaning

In [6]:
def raw_data_cleaning(raw_data):
  #raw_data = raw_data.drop(drop_columns, axis = 1)

  gps_data = raw_data[raw_data.latitude != 0]
  gps_data = gps_data[gps_data.longitude != 0] #cleaning zero values for latitude & longitude

  gps_data['date'] = pd.to_datetime(gps_data['devicetime']).dt.date #split date and time separately into datetime variables
  gps_data['time'] = pd.to_datetime(gps_data['devicetime']).dt.time
  gps_data['date'] = pd.to_datetime(gps_data['date'])
  
  gps_data = gps_data.sort_values(['date', 'time']) #sorting dataset by time and device

  # additional_columns = ['servertime','fixtime','address','routeid']

  # gps_data = gps_data.drop(columns=additional_columns, axis = 1) 

  gps_data.sort_values(by=['devicetime'], inplace=True) #sorting dataset by time and device

  # remove wrong dates
  gps_data = gps_data[gps_data['date'] >=  '2022-09-01']
  gps_data = gps_data[gps_data['date'] <= '2022-09-30'] 


  # drop 'date' and 'time' columns
  gps_data = gps_data.drop(columns=['date', 'time'], axis = 1)


  gps_data.reset_index(drop = True, inplace = True)

  return gps_data

In [7]:
gps_data = raw_data_cleaning(raw_data)

In [8]:
gps_data

,id,deviceid,devicetime,latitude,longitude,speed
0,1369679824,264,2022-09-01 00:03:29,7.296963,80.744453,0.0000
1,1369806540,250,2022-09-01 00:10:07,7.283123,80.697137,0.0000
2,1369690117,275,2022-09-01 00:13:11,7.292798,80.721103,0.0000
3,1369756885,121,2022-09-01 00:13:21,7.296603,80.715920,0.0000
4,1369679823,264,2022-09-01 00:18:29,7.296963,80.744453,0.0000
...,...,...,...,...,...,...
610291,1453607726,116,2022-09-30 23:58:58,8.206232,80.523298,35.0972
610292,1453607727,116,2022-09-30 23:59:13,8.203702,80.523393,35.0972
610293,1453607728,116,2022-09-30 23:59:28,8.201190,80.523505,35.0972
610294,1453607729,116,2022-09-30 23:59:43,8.198772,80.523610,31.8575


# preprocess bus terminals

In [9]:
bus_terminals = gpd.GeoDataFrame(bus_terminals, geometry=gpd.points_from_xy(bus_terminals.longitude,bus_terminals.latitude),crs='EPSG:4326') 

In [10]:
#  code to stop_id BT01 and BT02 and a 150m buffer and for other stop_id add 100m buffer using geometry.buffer variable
bus_terminals_buffer = bus_terminals.copy().to_crs('EPSG:5234')
bus_terminals_buffer['geometry'] = bus_terminals_buffer.geometry.buffer(100)


In [11]:
# divide bus_terminal into 2 datasets where first one has the first 16 rows and second one has the remianing rows
bus_terminals_KD = bus_terminals_buffer.iloc[1:15]
bus_terminals_DK = bus_terminals_buffer.iloc[17:]

# take only end terminals

In [12]:
end_terminal_buffer = bus_terminals_buffer[(bus_terminals_buffer['stop_id'] == 'BT01')|(bus_terminals_buffer['stop_id'] == 'BT02')]
end_terminal_buffer.geometry = end_terminal_buffer.geometry.buffer(150)

d:\Engineering\5th Semester\Project\myenv\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [13]:
bus_terminals_KD.head()

,stop_id,route_id,direction,address,latitude,longitude,geometry
1,101,654,Kandy-Digana,Wales Park,7.291186,80.637662,"POLYGON ((185076.961 232111.967, 185076.479 23..."
2,102,654,Kandy-Digana,Mahamaya,7.287840,80.645840,"POLYGON ((185979.853 231741.696, 185979.372 23..."
3,103,654,Kandy-Digana,Lewella junction,7.294430,80.650030,"POLYGON ((186442.694 232470.307, 186442.213 23..."
4,104,654,Kandy-Digana,Talwatta,7.286701,80.660336,"POLYGON ((187580.385 231615.322, 187579.903 23..."
5,105,654,Kandy-Digana,Tennekumbura Bridge,7.281866,80.666030,"POLYGON ((188209.005 231080.506, 188208.523 23..."


# Order by deviceid

In [14]:
# order gps data by deviceid 
gps_data = gps_data.sort_values(by=['deviceid','devicetime'])
gps_data.reset_index(drop = True, inplace = True)
gps_data

,id,deviceid,devicetime,latitude,longitude,speed
0,1370130062,116,2022-09-01 07:29:31,7.295148,80.735202,0.0000
1,1370130063,116,2022-09-01 07:29:46,7.295183,80.735237,0.0000
2,1370130058,116,2022-09-01 07:29:55,7.295187,80.735240,0.0000
3,1370130303,116,2022-09-01 07:29:56,7.295185,80.735238,0.0000
4,1370134540,116,2022-09-01 07:30:11,7.295197,80.735268,0.0000
...,...,...,...,...,...,...
610291,1450228238,1377,2022-09-29 19:18:27,7.271957,80.682667,28.0778
610292,1450228239,1377,2022-09-29 19:18:39,7.271170,80.683892,10.7991
610293,1450228240,1377,2022-09-29 19:18:54,7.270437,80.684927,25.3780
610294,1450228241,1377,2022-09-29 19:19:09,7.269738,80.686510,23.7581


# extract trip

In [15]:

def calculate_dwell_time(row, dwell, dwell_start, dwell_time):
    if row['speed'] == 0 and not dwell:
        dwell = True
        dwell_start = row['devicetime']
    elif dwell and (row['speed'] != 0):
        dwell = False
        dwell_end = row['devicetime']
        dwell_time += (dwell_end - dwell_start).total_seconds() / 60.0
    return dwell, dwell_start, dwell_time

In [16]:
#get the bus stop id of the current bus location
def get_bus_stop_id(point, bus_stops_gdf):
   
    for index, row in bus_stops_gdf.iterrows():
        if row['geometry'].contains(point):
            return row['stop_id']

    return None

In [17]:
#find the arrival and departure time of each bus stop
def find_arrival_departure_time(row,time_str, start_terminal,projected_point):

    global current_trip_id,date_str, bus_stop_id,new_bus_stop, bus_stop_data
    time = datetime.strptime(time_str, "%H:%M:%S")
    speed = row['speed']


    if start_terminal == 'BT01':
        new_bus_stop_id = get_bus_stop_id(projected_point, bus_terminals_buffer[bus_terminals_buffer['direction']=='Kandy-Digana'])
    else:
        new_bus_stop_id = get_bus_stop_id(projected_point, bus_terminals_buffer[bus_terminals_buffer['direction']=='Digana-Kandy'])
    

    if (bus_stop_id != new_bus_stop_id) and new_bus_stop_id:
        bus_stop_id = new_bus_stop_id
        new_bus_stop = {
            'deviceid': row['deviceid'],
            'trip_id': current_trip_id,
            'date': date_str,
            'bus_stop_id': bus_stop_id,
            'arrival_time': time,
            'arrival_time_0': None,
            'departure_time': None,
            'dwell_time': None
        }
        if speed == 0:
            new_bus_stop['arrival_time_0'] = time

    elif (bus_stop_id == new_bus_stop_id) and new_bus_stop_id:
        if speed == 0 and (not new_bus_stop['arrival_time_0']):
            new_bus_stop['arrival_time_0'] = time
            new_bus_stop['departure_time'] = None
            
        elif speed != 0 and (not new_bus_stop['departure_time']):
            new_bus_stop['departure_time'] = time


    elif (bus_stop_id != new_bus_stop_id) and (not new_bus_stop_id):
        bus_stop_id = new_bus_stop_id
        #set departure time
        if new_bus_stop['arrival_time_0'] and (not new_bus_stop['departure_time']):
            new_bus_stop['departure_time'] = time
        elif (new_bus_stop['arrival_time']) and (not new_bus_stop['departure_time']):
            new_bus_stop['departure_time'] = new_bus_stop['arrival_time']

        #set dwell time
        if new_bus_stop['arrival_time_0']:
            new_bus_stop['dwell_time'] = new_bus_stop['departure_time'] - new_bus_stop['arrival_time_0']
            new_bus_stop['arrival_time'] = new_bus_stop['arrival_time_0']
        else:
            new_bus_stop['dwell_time'] = new_bus_stop['departure_time'] - new_bus_stop['arrival_time']
        del new_bus_stop['arrival_time_0']

        new_bus_stop['arrival_time'] = new_bus_stop['arrival_time'].time()
        new_bus_stop['departure_time'] = new_bus_stop['departure_time'].time()

        bus_stop_data = pd.concat([bus_stop_data, pd.DataFrame([new_bus_stop])])
        bus_stop_data.to_csv('bus_stop_data_all.csv', index=False)


In [18]:
current_trip_id = 1
in_trip = False
date = None
start_time = None
start_terminal = None
end_time = None
end_terminal = None
travel_time = None
dwell_time = 0
inside = False
dwell = False
dwell_start = None
dwell_end = None
terminal = None
bus_stop_id = None
new_bus_stop_id = None
arrival_time = None
departure_time = None

In [24]:
# Initialize an empty DataFrame with column names
trip_data = pd.DataFrame(columns=['device_id','trip_id', 'date', 'start_time', 'end_time', 'start_terminal', 'end_terminal', 'travel_time', 'dwell_time'])
trip_data_rows = []

In [25]:
def process_row(row, end_terminal_buffer, bus_terminals_buffer):
    
    global current_trip_id,date_str, in_trip, date, start_time, start_terminal, end_time, end_terminal, travel_time, dwell_time, inside, dwell, dwell_start,trip_data,terminal, bus_stop_id, new_bus_stop_id,arrival_time,departure_time,bus_stop_data
    row['devicetime'] = pd.to_datetime(row['devicetime'])
    date_str = row['devicetime'].strftime('%Y-%m-%d')
    time_str = row['devicetime'].strftime('%H:%M:%S')
    point_geometry = Point(row['longitude'], row['latitude'])
    projected_point = gpd.GeoSeries([point_geometry], crs='EPSG:4326').to_crs(epsg=5234)[0]

    if end_terminal_buffer.contains(projected_point).any():
        
        terminal = 'BT01' if end_terminal_buffer.iloc[0].geometry.contains(projected_point) else 'BT02'
        inside = True
        if in_trip:
            if inside:
                in_trip = False
                end_time = time_str
                end_terminal = terminal
                travel_time = (datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).total_seconds() / 60.0
                
                if (start_terminal != end_terminal) and (travel_time > 0):
                    new_trip_row = {
                        'device_id': row['deviceid'], # 'deviceid': '123
                        'trip_id': current_trip_id,
                        'date': date,
                        'start_time': start_time,
                        'end_time': end_time,
                        'start_terminal': start_terminal,
                        'end_terminal': end_terminal,
                        'travel_time': travel_time,
                        'dwell_time': dwell_time
                    }
                    trip_data_rows.append(new_trip_row)
                    current_trip_id += 1

                else:
                    bus_stop_data=bus_stop_data[bus_stop_data['trip_id']!=current_trip_id]
                

                dwell_time = 0
                dwell = False

    elif not end_terminal_buffer.contains(projected_point).any() and inside:
        inside = False
        if not in_trip:
            in_trip = True
            date = date_str
            start_time = time_str
            start_terminal = terminal

    if in_trip:
        if ((start_terminal == 'BT01') and (not bus_terminals_buffer[bus_terminals_buffer['direction'] == 'Kandy-Digana'].contains(projected_point).any())) or ((start_terminal == 'BT02') and (not bus_terminals_buffer[bus_terminals_buffer['direction'] == 'Digana-Kandy'].contains(projected_point).any())):
            dwell, dwell_start, dwell_time = calculate_dwell_time(row, dwell, dwell_start, dwell_time)

        # find_arrival_departure_time(row,time_str,start_terminal,projected_point)


In [26]:
# Create empty DataFrames
bus_stop_data = pd.DataFrame(columns=['trip_id','date', 'bus_stop_id','arrival_time', 'departure_time','dwell_time'])

# Initialize variables
current_trip_id = current_trip_id = trip_data.iloc[-1]['trip_id'] + 1 if len(trip_data) > 0 else 1
in_trip = False
date = None
start_time = None
start_terminal = None
end_time = None
end_terminal = None
travel_time = None
dwell_time = 0
inside = False
dwell = False
dwell_start = None
dwell_end = None
terminal = None
bus_stop_id = None
new_bus_stop_id = None
arrival_time = None
departure_time = None

# Iterate through the rows of df123_11
for index, row in gps_data.iterrows():
    process_row(row, end_terminal_buffer, bus_terminals_buffer)

if trip_data_rows:
    trip_data = pd.concat([trip_data, pd.DataFrame(trip_data_rows)])
    trip_data.to_csv('trip_data_09_22.csv', index=False)


In [27]:
trip_data_rows[1]

{'device_id': 116,
 'trip_id': 2,
 'date': '2022-09-01',
 'start_time': '10:22:41',
 'end_time': '11:19:49',
 'start_terminal': 'BT01',
 'end_terminal': 'BT02',
 'travel_time': 57.13333333333333,
 'dwell_time': 6.366666666666666}

In [ ]:
trip_data.to_csv('trip_data_09_22.csv', index=False)

In [ ]:
# # code to map df123_11 and end_terminal_buffer using folium
# import folium
# from folium.plugins import MarkerCluster

# # Create a base map centered around the mean latitude and longitude of the data
# mean_latitude = df123_11['latitude'].mean()
# mean_longitude = df123_11['longitude'].mean()
# m = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=15)

# # Create a MarkerCluster for df123_11
# marker_cluster = MarkerCluster().add_to(m)

# # Add markers for each row in df123_11
# for index, row in df123_11.iterrows():
#     folium.Marker(
#         location=[row['latitude'], row['longitude']],
#         icon=None,
#         popup=f"Device ID: {row['deviceid']}<br>Time: {row['devicetime']}"
#     ).add_to(marker_cluster)

# # Add the end_terminal_buffer as a GeoJSON layer
# folium.GeoJson(
#     end_terminal_buffer,
#     name='Terminal Buffer',
#     style_function=lambda x: {'fillColor': 'blue', 'fillOpacity': 0.2}
# ).add_to(m)

# # Display the map
# m.save('trip_map.html')



In [ ]:
# filter device id 123
df123 = gps_data[gps_data.deviceid==123]
df123.reset_index(inplace=True)
df123.drop(columns=['index'], inplace=True)
df123